In [1]:
import wandb
import json
from datasets import Dataset, DatasetDict
import numpy as np

In [2]:
f = open(r"C:\Users\Leandro\Desktop\WandB_Projects\LLM_alpaca\alpaca_data_cleaned.json")
data = json.load(f)
f.close()

In [3]:
# Establecer la semilla para reproducibilidad
np.random.seed(42)

# Definir el rango de números (0 a 49999)
rango_numeros = np.arange(len(data))

# Generar lista_valid con 1000 números aleatorios sin repetición
lista_valid = np.random.choice(rango_numeros, size=1000, replace=False)

# Generar lista_train con los restantes 49000 números
lista_train = np.setdiff1d(rango_numeros, lista_valid)

In [4]:
# Establecer la semilla para reproducibilidad
np.random.seed(42)

# Definir el rango de números (0 a 49999)
rango_numeros = np.arange(len(data))

# Generar lista_valid con 1000 números aleatorios sin repetición
lista_valid = np.random.choice(rango_numeros, size=1000, replace=False)

# Generar lista_train con los restantes 49000 números
lista_train = np.setdiff1d(rango_numeros, lista_valid)

data_train = np.array(data)[lista_train]
data_valid = np.array(data)[lista_valid]

# Convertir la lista de diccionarios a un diccionario de listas
data_dict_train = {key: [item[key] for item in data_train] for key in data_train[0]}
data_dict_valid = {key: [item[key] for item in data_valid] for key in data_valid[0]}

# Crear un objeto Dataset a partir de la lista de diccionarios
custom_dataset_train = Dataset.from_dict(data_dict_train)
custom_dataset_valid = Dataset.from_dict(data_dict_valid)

# Convertir el Dataset en un DatasetDict
dataset_dict = DatasetDict({
    'train': custom_dataset_train,
    'valid': custom_dataset_valid})

In [5]:
train_dataset = dataset_dict['train']
eval_dataset = dataset_dict['valid']

In [6]:
def prompt_no_input(row):
    return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n{output}").format_map(row)

def prompt_input(row):
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n{output}").format_map(row)

def create_prompt(row):
    return prompt_no_input(row) if row["input"] == "" else prompt_input(row)

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [8]:
#model_id = 'openlm-research/open_llama_3b_v2'
#model_id = 'tiiuae/falcon-7b'
#model_id = 'bigscience/bloom-3b'
model_id = 'tiiuae/falcon-rw-1b'
#model_id = r'C:\Users\Leandro\.cache\huggingface\hub\models--bigscience--bloom-3b\snapshots\52bc5b43010b4844513826b8be3f78c7344c37d7'

In [9]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=64,  # the rank of the LoRA matrices
    lora_alpha=16, # the weight
    lora_dropout=0.1, # dropout to add to the LoRA layers
    bias="none", # add bias to the nn.Linear layers?
    task_type="CAUSAL_LM",
    #target_modules=["q_proj", "k_proj","v_proj","o_proj"], # the name of the layers to add LoRA (LLAMA 3B)
    #target_modules=['query_key_value'], # the name of the layers to add LoRA (Falcon 7B)
    target_modules=['query_key_value'], # the name of the layers to add LoRA (BLOOM 3B)
)

In [10]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [11]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [12]:
model_kwargs = dict(
    device_map=device,
    trust_remote_code=True,
    # low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    #use_flash_attention_2=True,
    use_cache=False,
    quantization_config=bnb_config,
)

In [13]:
from transformers import TrainingArguments
from trl import SFTTrainer

In [14]:
batch_size = 1
gradient_accumulation_steps = 16
num_train_epochs = 10

In [15]:
output_dir = "./output/falcon-1b"
training_args = TrainingArguments(
    num_train_epochs=num_train_epochs,
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    gradient_accumulation_steps=gradient_accumulation_steps,
    #gradient_checkpointing=True,
    gradient_checkpointing_kwargs=dict(use_reentrant=False),
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="epoch",
    report_to="wandb",
)

In [16]:
from utils import LLMSampleCB

trainer = SFTTrainer(
    model=model_id,
    model_init_kwargs=model_kwargs,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    packing=True,
    max_seq_length=1024,
    args=training_args,
    formatting_func=create_prompt,
    peft_config=peft_config,
)

c:\Users\Leandro\source\repos\PyTorchTest\PyTorchTest\TorchEnv\lib\site-packages\trl\trainer\sft_trainer.py:158: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


bin c:\Users\Leandro\source\repos\PyTorchTest\PyTorchTest\TorchEnv\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


In [17]:
#from peft import prepare_model_for_kbit_training
##trainer.model.config.use_cache = False
##trainer.model.gradient_checkpointing_enable()
#trainer.model = prepare_model_for_kbit_training(trainer.model)

In [18]:
# remove answers
def create_prompt_no_anwer(row):
    row["output"] = ""
    return {"text": create_prompt(row)}

test_dataset = eval_dataset.map(create_prompt_no_anwer)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
wandb_callback = LLMSampleCB(trainer, test_dataset, num_samples=30, max_new_tokens=256)

In [20]:
trainer.add_callback(wandb_callback)

In [21]:
with wandb.init(project='LLM_alpaca', name='falcon-1b'):
    trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: leandro-bello. Use `wandb login --relogin` to force relogin


  0%|          | 0/6230 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 2.1023, 'learning_rate': 3.210272873194222e-07, 'epoch': 0.0}
{'loss': 2.0459, 'learning_rate': 6.420545746388443e-07, 'epoch': 0.0}
{'loss': 1.9124, 'learning_rate': 9.630818619582665e-07, 'epoch': 0.0}
{'loss': 2.0229, 'learning_rate': 1.2841091492776887e-06, 'epoch': 0.01}
{'loss': 1.93, 'learning_rate': 1.6051364365971107e-06, 'epoch': 0.01}
{'loss': 2.0723, 'learning_rate': 1.926163723916533e-06, 'epoch': 0.01}
{'loss': 2.0631, 'learning_rate': 2.247191011235955e-06, 'epoch': 0.01}
{'loss': 2.0747, 'learning_rate': 2.5682182985553774e-06, 'epoch': 0.01}
{'loss': 2.0835, 'learning_rate': 2.8892455858747994e-06, 'epoch': 0.01}
{'loss': 1.9656, 'learning_rate': 3.2102728731942214e-06, 'epoch': 0.02}
{'loss': 1.9379, 'learning_rate': 3.531300160513644e-06, 'epoch': 0.02}
{'loss': 1.9743, 'learning_rate': 3.852327447833066e-06, 'epoch': 0.02}
{'loss': 1.9185, 'learning_rate': 4.173354735152488e-06, 'epoch': 0.02}
{'loss': 1.8956, 'learning_rate': 4.49438202247191e-06, 'epoch':

  0%|          | 0/201 [00:00<?, ?it/s]

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

{'loss': 1.2773, 'learning_rate': 0.00019999998430329032, 'epoch': 1.0}
{'loss': 1.2079, 'learning_rate': 0.00019999993721316617, 'epoch': 1.0}
{'loss': 1.2171, 'learning_rate': 0.00019999985872964232, 'epoch': 1.0}
{'loss': 1.2262, 'learning_rate': 0.00019999974885274348, 'epoch': 1.01}
{'loss': 1.1884, 'learning_rate': 0.00019999960758250408, 'epoch': 1.01}
{'loss': 1.2091, 'learning_rate': 0.00019999943491896848, 'epoch': 1.01}
{'loss': 1.2155, 'learning_rate': 0.0001999992308621909, 'epoch': 1.01}
{'loss': 1.2788, 'learning_rate': 0.0001999989954122354, 'epoch': 1.01}
{'loss': 1.2888, 'learning_rate': 0.0001999987285691759, 'epoch': 1.01}
{'loss': 1.2152, 'learning_rate': 0.00019999843033309612, 'epoch': 1.01}
{'loss': 1.2881, 'learning_rate': 0.00019999810070408974, 'epoch': 1.02}
{'loss': 1.2868, 'learning_rate': 0.00019999773968226023, 'epoch': 1.02}
{'loss': 1.2385, 'learning_rate': 0.00019999734726772092, 'epoch': 1.02}
{'loss': 1.2182, 'learning_rate': 0.00019999692346059502,

Traceback (most recent call last):
  File "C:\Users\Leandro\AppData\Local\Temp\ipykernel_25032\3042643859.py", line 2, in <module>
    trainer.train()
  File "c:\Users\Leandro\source\repos\PyTorchTest\PyTorchTest\TorchEnv\lib\site-packages\trl\trainer\sft_trainer.py", line 315, in train
    output = super().train(*args, **kwargs)
  File "c:\Users\Leandro\source\repos\PyTorchTest\PyTorchTest\TorchEnv\lib\site-packages\transformers\trainer.py", line 1537, in train
    return inner_training_loop(
  File "c:\Users\Leandro\source\repos\PyTorchTest\PyTorchTest\TorchEnv\lib\site-packages\transformers\trainer.py", line 1854, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "c:\Users\Leandro\source\repos\PyTorchTest\PyTorchTest\TorchEnv\lib\site-packages\transformers\trainer.py", line 2744, in training_step
    self.accelerator.backward(loss)
  File "c:\Users\Leandro\source\repos\PyTorchTest\PyTorchTest\TorchEnv\lib\site-packages\accelerate\accelerator.py", li

eval/loss,▁▁
eval/runtime,▁▁
eval/samples_per_second,▁▁
eval/steps_per_second,▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██████
train/loss,█▇▇▆▅▅▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▃▂▁▂▂▂▂▂▂▂▁▁▂▂▂▂
eval/loss,1.24531
eval/runtime,143.0478
eval/samples_per_second,1.405


KeyboardInterrupt: 